In [1]:
import math
import tsv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

root_path = "../data/poly_mo/"

# preprocess data files

# Quant_bootstraps.tsv :containing the matrix of bootstrap experiments 
# containing the final count for each transcript in each round of bootstrapping 
# with a row be a bootstrap output and columns be list of transcripts. 
quant_bootstraps = tsv.TsvReader(open(root_path+"quant_bootstraps.tsv"))
count = 0
quant_boot = []
for parts in quant_bootstraps:
    quant_boot.append(parts)
df_quant_boot = pd.DataFrame.from_records(quant_boot[1:], columns=quant_boot[0])
df_quant_boot = df_quant_boot.astype('float')
df_quant_boot_mean = df_quant_boot.mean()
df_quant_boot_std = df_quant_boot.std()
id_in_quant_boot = list(df_quant_boot.columns)

# Quant.sf :estimated quantifications for each transcript
quant_file = open(root_path+"quant.sf")
lines = quant_file.readlines()
quant_file.close()
count = 0
quant = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    quant.append(l)

df_quant = pd.DataFrame.from_records(quant[1:], columns=quant[0])
df_quant.Name = df_quant.Name.astype(str)
df_quant.Length = df_quant.Length.astype(int)
df_quant.EffectiveLength = df_quant.EffectiveLength.astype(float)
df_quant.TPM = df_quant.TPM.astype(float)
df_quant.NumReads = df_quant.NumReads.astype(float)

# eq_classes


In [2]:
fixed_dfquant_boot = df_quant_boot.copy()
columns = fixed_dfquant_boot.columns

sort_qb = []
for id in columns:
    try:
        listed = list(df_quant_boot[id])        
    except KeyError:
        pass
    else:
        listed.sort()
        sort_qb.append(listed)
sort_qb = list(map(list,zip(*sort_qb)))

df_qb_sorted = pd.DataFrame.from_records(sort_qb, columns=columns)

sum = len(sort_qb)
print(int(sum*0.025))
print(int(sum*0.975))
percent2dot5 = df_qb_sorted.loc[int(sum*0.025)-1]
percent97dot5 = df_qb_sorted.loc[int(sum*0.975)-1]

remove_ids = []
for id in columns:
    if float(percent97dot5[id]) == 0:
        remove_ids.append(id)

new_columns = list(set(columns) - set(remove_ids))
fixed_dfquant_boot = fixed_dfquant_boot[new_columns] 
fixed_dfquant_boot = fixed_dfquant_boot.astype('float')

5
195


In [3]:
fixed_dfquant = df_quant.copy()
print(len(fixed_dfquant.Name)) # 93109
for rid in remove_ids:  # too slow
    fixed_dfquant = fixed_dfquant[fixed_dfquant.Name != rid]
print(len(fixed_dfquant.Name)) # 33965

# we now have:
# fixed_dfquant_boot from df_quant_boot
# fixed_dfquant from from df_quant
# mean: fixed_dfquant_boot.mean()
# std:  fixed_dfquant_boot.std()
# fixed_dfquant.Name
# fixed_dfquant.Length
# fixed_dfquant.EffectiveLength
# fixed_dfquant.TPM
# fixed_dfquant.NumReads

df_attributes = fixed_dfquant.copy()
fixed_qb_mean = fixed_dfquant_boot.mean()
fixed_qb_std = fixed_dfquant_boot.std()
fixed_qb_mean = fixed_qb_mean.tolist()
fixed_qb_std = fixed_qb_std.tolist()

df_attributes.insert(5,'quant_boot_mean', fixed_qb_mean)
df_attributes.insert(6,'quant_boot_std', fixed_qb_std)

93109
33965


In [4]:
df_attributes.describe()

,Length,EffectiveLength,TPM,NumReads,quant_boot_mean,quant_boot_std
count,33965.000000,33965.000000,33965.000000,3.396500e+04,3.396500e+04,3.396500e+04
mean,2262.225438,2063.146050,29.442073,9.811421e+02,9.811410e+02,2.294763e+01
std,2381.743492,2381.672015,332.764064,1.143815e+04,1.143812e+04,4.136096e+01
min,82.000000,9.804000,0.000000,0.000000e+00,5.453287e-10,3.365153e-09
25%,755.000000,555.999000,0.085157,2.939223e+00,3.613455e+00,2.956872e+00
50%,1628.000000,1428.991000,1.237854,4.700000e+01,4.714089e+01,1.071328e+01
75%,2946.000000,2746.991000,8.238800,3.988340e+02,3.978234e+02,2.839883e+01
max,104301.000000,104101.991000,23356.420222,1.109005e+06,1.109140e+06,1.112645e+03


In [5]:
import keras

ModuleNotFoundError: No module named 'keras'